# Example of use of Latent Class MNL

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artefactory/choice-learn/blob/main/notebooks/models/latent_class_model.ipynb)

In [1]:
# Install necessary requirements

# If you run this notebook on Google Colab, or in standalone mode, you need to install the required packages.
# Uncomment the following lines:

# !pip install choice-learn

# If you run the notebook within the GitHub repository, you need to run the following lines, that can skipped otherwise:
import os
import sys

sys.path.append("../../")

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import matplotlib as mpl
import numpy as np
import pandas as pd

import tensorflow as tf

2024-12-23 10:04:07.728988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let's use the Electricity Dataset used in this [tutorial](https://rpubs.com/msarrias1986/335556).

In [3]:
from choice_learn.datasets import load_electricity

elec_dataset = load_electricity(as_frame=False)

In [4]:
from choice_learn.models.simple_mnl import SimpleMNL
from choice_learn.models.latent_class_mnl import LatentClassSimpleMNL

In [5]:
lc_model = LatentClassSimpleMNL(n_latent_classes=3, fit_method="mle", optimizer="lbfgs", epochs=1000, lbfgs_tolerance=1e-20)
hist, results = lc_model.fit(elec_dataset, verbose=1)

Using L-BFGS optimizer, setting up .fit() function
Using L-BFGS optimizer, setting up .fit() function
Using L-BFGS optimizer, setting up .fit() function
L-BFGS Opimization finished:
---------------------------------------------------------------
Number of iterations: 202
Algorithm converged before reaching max iterations: True


In [6]:
print("Latent Class Model weights:")
print("Classes Logits:", lc_model.latent_logits)
for i in range(3):
    print("\n")
    print(f"Model Nb {i}, weights:", lc_model.models[i].weights)

Latent Class Model weights:
Classes Logits: <tf.Variable 'Latent-Logits:0' shape=(2,) dtype=float32, numpy=array([-2.1254017, -2.759119 ], dtype=float32)>


Model Nb 0, weights: [<tf.Variable 'Weights_items_features:0' shape=(6,) dtype=float32, numpy=
array([-0.6756455 , -0.06060429,  1.8519509 ,  1.3225493 , -5.8570886 ,
       -6.513206  ], dtype=float32)>]


Model Nb 1, weights: [<tf.Variable 'Weights_items_features:0' shape=(6,) dtype=float32, numpy=
array([ -1.8175662,  -1.7263651,   3.6965666,   4.1118402, -26.693516 ,
       -14.92584  ], dtype=float32)>]


Model Nb 2, weights: [<tf.Variable 'Weights_items_features:0' shape=(6,) dtype=float32, numpy=
array([ -2.1047907,  -1.6526219,  -5.5542874, -13.565555 ,  -9.79493  ,
       -12.126673 ], dtype=float32)>]


In [7]:
nll = (lc_model.evaluate(elec_dataset) * len(elec_dataset)).numpy()
print(f"Negative Log-Likelihood: {nll}")

Negative Log-Likelihood: 4920.5439453125


In [19]:
report = lc_model.compute_report(elec_dataset)

def format_color_groups(df):
    cmap = mpl.cm.get_cmap("Set1")
    colors = [mpl.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]
    x = df.copy()
    factors = list(x['Latent Class'].unique())
    i = 0
    for factor in factors:
        style = f'background-color: {colors[i]}'
        x.loc[x['Latent Class'] == factor, :] = style
        i += 1
    return x

report.style.apply(format_color_groups, axis=None)

Using L-BFGS optimizer, setting up .fit() function
Using L-BFGS optimizer, setting up .fit() function
Using L-BFGS optimizer, setting up .fit() function


/var/folders/zz/r1py7zhj35q75v09h8_42nzh0000gp/T/ipykernel_67121/1263996749.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap("Set1")


,Latent Class,Coefficient Name,Coefficient Estimation,Std. Err,z_value,P(.>z)
0,0,Weights_items_features_0,-0.675645,0.023987,-28.167109,0.000000
1,0,Weights_items_features_1,-0.060604,0.008162,-7.424849,0.000000
2,0,Weights_items_features_2,1.851951,0.054914,33.724579,0.000000
3,0,Weights_items_features_3,1.322549,0.048159,27.462420,0.000000
4,0,Weights_items_features_4,-5.857089,0.191162,-30.639460,0.000000
5,0,Weights_items_features_5,-6.513206,0.195680,-33.285046,0.000000
6,1,Weights_items_features_0,-1.817566,0.077771,-23.370796,0.000000
7,1,Weights_items_features_1,-1.726365,0.058838,-29.340986,0.000000
8,1,Weights_items_features_2,3.696567,0.160258,23.066404,0.000000
9,1,Weights_items_features_3,4.111840,0.157179,26.160225,0.000000


## Latent Conditional Logit
We used a very simple MNL. Here we simulate the same MNL, by using the Conditional-Logit formulation.\
Don't hesitate to read the conditional-MNL tutorial to better understand how to use this formulation.

In [ ]:
from choice_learn.models.latent_class_mnl import LatentClassConditionalLogit

In [ ]:
lc_model_2 = LatentClassConditionalLogit(n_latent_classes=3,
                                       fit_method="mle",
                                       optimizer="lbfgs",
                                       epochs=1000,
                                       lbfgs_tolerance=1e-12)

For each feature, let's add a coefficient that is shared by all items:

In [ ]:
lc_model_2.add_shared_coefficient(coefficient_name="pf",
                                  feature_name="pf",
                                  items_indexes=[0, 1, 2, 3])
lc_model_2.add_shared_coefficient(coefficient_name="cl",
                                  feature_name="cl",
                                    items_indexes=[0, 1, 2, 3])
lc_model_2.add_shared_coefficient(coefficient_name="loc",
                                  feature_name="loc",
                                  items_indexes=[0, 1, 2, 3])
lc_model_2.add_shared_coefficient(coefficient_name="wk",
                                feature_name="wk",
                                items_indexes=[0, 1, 2, 3])
lc_model_2.add_shared_coefficient(coefficient_name="tod",
                                  feature_name="tod",
                                  items_indexes=[0, 1, 2, 3])
lc_model_2.add_shared_coefficient(coefficient_name="seas",
                                  feature_name="seas",
                                  items_indexes=[0, 1, 2, 3])

In [ ]:
# Fit
hist2 = lc_model_2.fit(elec_dataset, verbose=1)

In [ ]:
print("Negative Log-Likelihood:", lc_model_2.evaluate(elec_dataset)*len(elec_dataset))

In [ ]:
print("Latent Class Model weights:")
print("Classes Logits:", lc_model_2.latent_logits)
for i in range(3):
    print("\n")
    print(f"Model Nb {i}, weights:", lc_model_2.models[i].trainable_weights)

Just like any ChoiceModel you can get the probabilities:

In [ ]:
lc_model.predict_probas(elec_dataset[:4])

If you want to use more complex formulations of Latent Class models, you can directly use the *BaseLatentClassModel* from *choice_learn.models.base_model*:

In [ ]:
from choice_learn.models.latent_class_base_model import BaseLatentClassModel

In [ ]:
manual_lc = BaseLatentClassModel(
                                 model_class=SimpleMNL,
                                 model_parameters={"add_exit_choice": False},
                                 n_latent_classes=3,
                                 fit_method="mle",
                                 epochs=1000,
                                 optimizer="lbfgs",
                                 lbfgs_tolerance=1e-12
                                 )
manual_lc.instantiate(n_items=4,
                      n_shared_features=0,
                      n_items_features=6)
manual_hist = manual_lc.fit(elec_dataset, verbose=1)

In [ ]:
print(manual_lc.evaluate(elec_dataset) * len(elec_dataset))

If you need to go deeper, you can look [here](../choice_learn/models/latent_class_mnl.py) to see different implementations that could help you.